In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
DIRECTORY = './data/'
categories = pd.read_csv(DIRECTORY + 'item_categories.csv')
items = pd.read_csv(DIRECTORY + 'items.csv')
shops = pd.read_csv(DIRECTORY + 'shops.csv')
train = pd.read_csv(DIRECTORY + 'sales_train.csv')

# Backtest Engine

In [5]:
class BackTester:
    def __init__(self, train):
        date_train = train.copy()
        date_train['date'] = pd.to_datetime(date_train['date'], format='%d.%m.%Y')
        date_train['month'] = date_train['date'].dt.to_period('M')
        date_train = date_train.set_index('month')
        date_train = date_train.sort_index()

        self.date_train = self.

    months = date_train.index.unique()
    for index, month in enumerate(months):
        if index == len(months) - 1: continue # exclude last month (using full train set)
        if index > 1: continue

        sub_train = date_train.loc[:month, :]
        sub_test = date_train.loc[months[index+1],:]



,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
month,,,,,,
2013-01,2013-01-02,0,59,22154,999.00,1.0
2013-01,2013-01-03,0,25,2552,899.00,1.0
2013-01,2013-01-05,0,25,2552,899.00,-1.0
2013-01,2013-01-06,0,25,2554,1709.05,1.0
2013-01,2013-01-15,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2013-01,2013-01-19,0,46,32,349.00,1.0
2013-01,2013-01-18,0,46,32,349.00,1.0
2013-01,2013-01-26,0,46,35,399.00,2.0


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
month,,,,,,
2013-02,2013-02-21,1,50,3880,1499.0,1.0
2013-02,2013-02-14,1,50,3880,1499.0,1.0
2013-02,2013-02-21,1,50,4128,899.0,1.0
2013-02,2013-02-13,1,50,4124,249.0,1.0
2013-02,2013-02-24,1,50,3880,1499.0,1.0
...,...,...,...,...,...,...
2013-02,2013-02-23,1,28,13511,1599.0,2.0
2013-02,2013-02-26,1,28,13511,1599.0,1.0
2013-02,2013-02-18,1,28,13413,1999.0,1.0


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
month,,,,,,
2013-01,2013-01-02,0,59,22154,999.00,1.0
2013-01,2013-01-03,0,25,2552,899.00,1.0
2013-01,2013-01-05,0,25,2552,899.00,-1.0
2013-01,2013-01-06,0,25,2554,1709.05,1.0
2013-01,2013-01-15,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2013-02,2013-02-23,1,28,13511,1599.00,2.0
2013-02,2013-02-26,1,28,13511,1599.00,1.0
2013-02,2013-02-18,1,28,13413,1999.00,1.0


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
month,,,,,,
2013-03,2013-03-03,2,5,20175,849.0,1.0
2013-03,2013-03-06,2,5,20083,149.0,1.0
2013-03,2013-03-02,2,2,31,698.5,1.0
2013-03,2013-03-17,2,41,2437,699.5,1.0
2013-03,2013-03-02,2,41,2417,349.0,1.0
...,...,...,...,...,...,...
2013-03,2013-03-02,2,5,4163,1590.0,1.0
2013-03,2013-03-16,2,5,4072,299.0,1.0
2013-03,2013-03-30,2,5,3686,675.0,1.0
